In [1]:
import pandas as pd
import numpy as np
import random

randomlist = []
df_demand = {}
random.seed(22) # Use the same random seed
for i in range(0,50):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(50):
    df_demand[i] = randomlist[i]

sum = 0
for (k,v) in df_demand.items():
    sum = sum + v
    
df_supply = {}
for i in range(sum):
    df_supply[i] = 1.0
    
np.random.seed(22) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,60, size=(sum,50))).astype(float)

random.seed(22) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)
    
age = {}
for i in range(len(age_b)):
    age[i] = age_b[i] 
    
kpl = []
co2 = []
random.seed(22) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)
        
fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])
    
random.seed(22) # Use the same random seed
doc = {}
for i in range(0,50):
    n = random.uniform(50,100)
    doc[i] = n
    
df_tdkom_c = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom_c[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom_c

In [2]:
df_cost = df_tdkom_c.copy(deep=True)

# Calculating the row deadkm opportunity matrix 
df_rowmin = df_cost.min(axis=1)
df_rowopp = df_cost.copy(deep=True)
for i in range(0,len(df_cost)):
    for j in range(0,len(df_cost.columns)):
        df_rowopp.iloc[i,j] = df_rowopp.iloc[i,j] - df_rowmin.iloc[i]

# Calculating the column deadkm opportunity matrix 
df_colmin = df_cost.min(axis=0)
df_colopp = df_cost.copy(deep=True)
for i in range(0,len(df_cost.columns)):
    for j in range(0,len(df_cost)):
        df_colopp.iloc[j,i] = df_colopp.iloc[j,i] - df_colmin.iloc[i]

# Calculating the total deadkm opportunity matrix 
df_tdkom = round(df_rowopp + df_colopp, 2)
#print("Total Deadkm opportunity matrix is:")
#print(df_tdkom)


df_res = df_tdkom.copy(deep=True)
df_result = pd.DataFrame(np.zeros(df_tdkom.shape))

In [ ]:
while(list(df_supply.values()) and list(df_demand.values())): 
    
    # To find the minimum cost row-wise
    df_nr_list = list(df_tdkom.min(axis=1))
    df_nr = {}
    for i in range(len(df_tdkom.index)):
        df_nr[df_tdkom.index[i]] = df_nr_list[i]
    #df_nr

    # To find the minimum cost column-wise
    df_nc_list = list(df_tdkom.min(axis=0))
    df_nc = {}
    for i in range(len(df_tdkom.columns)):
        df_nc[df_tdkom.columns[i]] = df_nc_list[i]
    #df_nc

    import numpy as np
    stdr = {}
    for i in range(len(df_tdkom.index)):
        stdr[df_tdkom.index[i]] = list(np.std(df_tdkom, axis = 1))[i]
    #stdr

    stdc = {}
    for i in range(len(df_tdkom.columns)):
        stdc[df_tdkom.columns[i]] = list(np.std(df_tdkom, axis = 0))[i]
    #stdc

    r = max(list(stdr.values()) + list(stdc.values()))
    #r

    lst = []
    if(r in stdc.values() and r not in stdr.values()):
        for i in df_tdkom.columns:
            if (stdc[i] == r): 
                for j in (df_tdkom.index):
                    if (df_tdkom.loc[j,i] == df_nc[i]):
                        lst.append([j,i])
                        
        if (len(lst) == 1):
            df_result.loc[lst[0][0],lst[0][1]] = min(df_supply[lst[0][0]],df_demand[lst[0][1]])
            df_supply[lst[0][0]] = df_supply[lst[0][0]] - df_result.loc[lst[0][0],lst[0][1]]
            df_demand[lst[0][1]] = df_demand[lst[0][1]] - df_result.loc[lst[0][0],lst[0][1]]


            if (df_supply[lst[0][0]] == 0 and df_demand[lst[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lst[0][0], axis = 0)
                del df_supply[lst[0][0]]

            elif (df_supply[lst[0][0]] == 0 and df_demand[lst[0][1]] != 0):
                df_tdkom = df_tdkom.drop(lst[0][0], axis = 0)
                del df_supply[lst[0][0]]

            elif (df_supply[lst[0][0]] != 0 and df_demand[lst[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lst[0][1], axis = 1)
                del df_demand[lst[0][1]]
        
        elif (len(lst)!=1):
            minc = df_tdkom.loc[lst[0][0],lst[0][1]]
            for i in range(len(lst)):
                if (minc <= df_tdkom.loc[lst[i][0],lst[i][1]]):
                    minc = minc
                elif (minc > df_tdkom.loc[lst[i][0],lst[i][1]]):
                    minc = df_tdkom.loc[lst[i][0],lst[i][1]]
                    
            lstn = []
            for i in range(len(lst)):
                if (df_tdkom.loc[lst[i][0],lst[i][1]] == minc):
                    lstn.append([lst[i][0],lst[i][1]])

            df_result.loc[lstn[0][0],lstn[0][1]] = min(df_supply[lstn[0][0]],df_demand[lstn[0][1]])
            df_supply[lstn[0][0]] = df_supply[lstn[0][0]] - df_result.loc[lstn[0][0],lstn[0][1]]
            df_demand[lstn[0][1]] = df_demand[lstn[0][1]] - df_result.loc[lstn[0][0],lstn[0][1]]
            
            if (df_supply[lstn[0][0]] == 0 and df_demand[lstn[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lstn[0][0], axis = 0)
                del df_supply[lstn[0][0]]

            elif (df_supply[lstn[0][0]] == 0 and df_demand[lstn[0][1]] != 0):
                df_tdkom = df_tdkom.drop(lstn[0][0], axis = 0)
                del df_supply[lstn[0][0]]

            elif (df_supply[lstn[0][0]] != 0 and df_demand[lstn[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lstn[0][1], axis = 1)
                del df_demand[lstn[0][1]]


    lstm = [] 
    lstr = [] 
    #### Tie in value of SD, selecting minimum TC
    ### Only if same SD appears row-wise and once column-wise
    if(r in stdc.values() and r in stdr.values()):
        min_val = []
        idx_list = []
        for i in df_tdkom.index:
            for j in (df_tdkom.columns):
                if (stdr[i] == r and stdc[j] == r):
                    if (df_nr[i] <= df_nc[j]):
                        for l in df_tdkom.columns:
                            if (df_tdkom.loc[i,l] == df_nr[i]):
                                #print(df_tdkom.loc[i,l], df_nr[i])
                                min_val.append(df_nr[i])
                                idx_list.append([i,l])
                                #lstm.append([i,l])
                    elif (df_nr[i] > df_nc[j]):
                        for o in df_tdkom.index:
                             if (df_tdkom.loc[o,j] == df_nc[j]):
                                    min_val.append(df_nc[j])
                                    idx_list.append([o,j])
                                    #lstm.append([o,j]) 
                    lstm.append(idx_list[min_val.index(min(min_val))])         
 

        ###### Allocating to the left most cell, since that' how it identifies
        df_result.loc[lstm[0][0],lstm[0][1]] = min(df_supply[lstm[0][0]],df_demand[lstm[0][1]])
        df_supply[lstm[0][0]] = df_supply[lstm[0][0]] - df_result.loc[lstm[0][0],lstm[0][1]]
        df_demand[lstm[0][1]] = df_demand[lstm[0][1]] - df_result.loc[lstm[0][0],lstm[0][1]]
        
        
        if (df_supply[lstm[0][0]] == 0 and df_demand[lstm[0][1]] == 0):
            #print(df_supply[lstm[0][0]], df_demand[lstm[0][1]] )
            df_tdkom = df_tdkom.drop(lstm[0][0], axis = 0)
            del df_supply[lstm[0][0]]
            
        elif (df_supply[lstm[0][0]] == 0 and df_demand[lstm[0][1]] != 0):
            #print(df_supply[lstm[0][0]], df_demand[lstm[0][1]] )
            df_tdkom = df_tdkom.drop(lstm[0][0], axis = 0)
            del df_supply[lstm[0][0]]
        
        elif (df_supply[lstm[0][0]] != 0 and df_demand[lstm[0][1]] == 0):
            #print(df_supply[lstm[0][0]], df_demand[lstm[0][1]] )
            df_tdkom = df_tdkom.drop(lstm[0][1], axis = 1)
            del df_demand[lstm[0][1]]

   
    if (r in stdr.values() and r not in stdc.values()):
        for i in df_tdkom.index:
            if (stdr[i] == r): 
                for j in (df_tdkom.columns):
                    if (df_tdkom.loc[i,j] == df_nr[i]):
                        lstr.append([i,j])
        
        if (len(lstr)==1):
            df_result.loc[lstr[0][0],lstr[0][1]] = min(df_supply[lstr[0][0]],df_demand[lstr[0][1]])
            df_supply[lstr[0][0]] = df_supply[lstr[0][0]] - df_result.loc[lstr[0][0],lstr[0][1]]
            df_demand[lstr[0][1]] = df_demand[lstr[0][1]] - df_result.loc[lstr[0][0],lstr[0][1]]  

            if (df_supply[lstr[0][0]] == 0 and df_demand[lstr[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lstr[0][0], axis = 0)
                del df_supply[lstr[0][0]]

            elif (df_supply[lstr[0][0]] == 0 and df_demand[lstr[0][1]] != 0):
                df_tdkom = df_tdkom.drop(lstr[0][0], axis = 0)
                del df_supply[lstr[0][0]]

            elif (df_supply[lstr[0][0]] != 0 and df_demand[lstr[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lstr[0][1], axis = 1)
                del df_demand[lstr[0][1]]
                
        elif (len(lstr)!=1):
            minr = df_tdkom.loc[lstr[0][0],lstr[0][1]]
            for i in range(len(lstr)):
                if (minr <= df_tdkom.loc[lstr[i][0],lstr[i][1]]):
                    minr = minr
                elif (minr > df_tdkom.loc[lstr[i][0],lstr[i][1]]):
                    minr = df_tdkom.loc[lstr[i][0],lstr[i][1]]

            lstp = []
            for i in range(len(lstr)):
                if (df_tdkom.loc[lstr[i][0],lstr[i][1]] == minr):
                    lstp.append([lstr[i][0],lstr[i][1]])

            df_result.loc[lstp[0][0],lstp[0][1]] = min(df_supply[lstp[0][0]],df_demand[lstp[0][1]])
            df_supply[lstp[0][0]] = df_supply[lstp[0][0]] - df_result.loc[lstp[0][0],lstp[0][1]]
            df_demand[lstp[0][1]] = df_demand[lstp[0][1]] - df_result.loc[lstp[0][0],lstp[0][1]]

            if (df_supply[lstp[0][0]] == 0 and df_demand[lstp[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lstp[0][0], axis = 0)
                del df_supply[lstp[0][0]]

            elif (df_supply[lstp[0][0]] == 0 and df_demand[lstp[0][1]] != 0):
                df_tdkom = df_tdkom.drop(lstp[0][0], axis = 0)
                del df_supply[lstp[0][0]]

            elif (df_supply[lstp[0][0]] != 0 and df_demand[lstp[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lstp[0][1], axis = 1)
                del df_demand[lstp[0][1]]


In [ ]:
df_result

In [ ]:
df_final = df_cost.mul(df_result, axis = 0)
df_final.to_numpy().sum()

stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time))